In [ ]:
import rasterio
import numpy as np

# Data Source - https://github.com/globalmaps/gm_ve_v1
forests_file = rasterio.open('../resources/data/downsampled_forest.tif')
forests = forests_file.read()

print(np.amin(forests))
print(np.amax(forests))
print(len(np.unique(forests)))

In [ ]:
from rasterio.plot import show
import matplotlib.pyplot as plt

show(forests, cmap='Greens')

In [ ]:
from matplotlib.colors import ListedColormap, LinearSegmentedColormap, BoundaryNorm
from matplotlib import cm

greens = cm.get_cmap('Greens', 100)
newcolors = greens(np.linspace(0, 1, 100))
background_colour = np.array([0.0, 0.0, 0.0, 0.0])
newcolors[:1, :] = background_colour
newcmp_forests = ListedColormap(newcolors)

bounds = np.arange(100)
norm_forests = BoundaryNorm(bounds, newcmp_forests.N)

In [ ]:
show(forests, ax=ax, cmap=newcmp_forests, norm=norm_forests)

In [ ]:
import rioxarray as rxr
from rasterio.crs import CRS
import numpy as np

# Load raster and squeeze singleton dimensions
forests_file_rxa = rxr.open_rasterio('../resources/data/downsampled_forest.tif', masked=True).squeeze()

print("Original projection: ", forests_file_rxa.rio.crs)

# Reproject to new CRS
forests = forests_file_rxa.rio.reproject(CRS.from_string('+proj=robin'))

print("New projection: ", forests.rio.crs)

# Convert to NumPy array
forests_np = forests.to_numpy()

# Mask nodata values and set them to zero
nodata_value = np.finfo(np.float32).max  # 3.4028235e+38
forests_np[forests_np == nodata_value] = 0

## Exercise

- Plot the reprojected raster
- Pick a new - interesting projection and reproject your raster to that. 

<img src="../resources/forests_rob.png">

In [ ]:
import geopandas as gpd
import pandas as pd

# Data source - https://github.com/nvkelso/natural-earth-vector/blob/master/10m_cultural/ne_10m_admin_0_countries.shp
df = gpd.read_file('../resources/data/ne_10m_admin_0_countries.shp')

brazil = df.loc[df['ADMIN'] == 'Brazil']
brazil.plot()

In [ ]:
import rasterio
from rasterio import mask as msk 
from shapely.geometry import mapping

clipped_array, clipped_transform = rasterio.mask.mask(forests_file, [mapping(geom) for geom in (brazil.geometry.tolist())], 
                                                      crop=True, nodata=0)

## Exercise 

- Plot the forests map of Brazil. 
- Use the topography dataset - ../resources/downsampled_topography.tif and generate a topography map of a country.

In [ ]:
from rasterio.io import MemoryFile

memfile = MemoryFile()
dataset = memfile.open(driver='GTiff', height=clipped_array.shape[1], width=clipped_array.shape[2], 
                       count=1, crs=forests_file.crs, 
                       transform=clipped_transform, dtype=clipped_array.dtype)
dataset.write(clipped_array[0], 1)

In [ ]:
from rasterio.enums import Resampling
from rasterio.warp import calculate_default_transform, reproject

# Define the destination CRS - Mercator projection
dst_crs = 'EPSG:3395'

# Calculate the transform and dimensions for the new projection
transform, width, height = calculate_default_transform(dataset.crs, dst_crs, dataset.width, dataset.height, *dataset.bounds)

# Create a destination numpy array
dst_array = np.empty((height, width), dtype=clipped_array.dtype)

# Perform the reprojection
reproject(
    source=rasterio.band(dataset, 1),
    destination=dst_array,
    src_transform=dataset.transform,
    src_crs=dataset.crs,
    dst_transform=transform,
    dst_crs=dst_crs,
    resampling=Resampling.nearest
)

In [ ]:
show(dst_array, transform=transform, cmap=newcmp_forests, norm=norm_forests)

In [ ]:
# Plot raster
fig, (ax1, ax2) = plt.subplots(nrows=1, ncols=2, figsize=(10, 4))

show(clipped_array, ax=ax1, transform=clipped_transform, cmap=newcmp_forests, norm=norm_forests)

show(dst_array, ax=ax2, transform=transform, cmap=newcmp_forests, norm=norm_forests)

plt.show()

## Exercise 

- Plot both the rivers from the previous exercise and forests from this exercise on the same map. 
- Try out different projections - but make sure all datasets have the same projection.